In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from cooccurrence import *
from glob import glob
from time import time
import helper
import sys
import networkx as nx
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import dataset_helper
import preprocessing
import os
import graph_helper

In [15]:
%%time
X_all, Y_all = dataset_helper.get_dataset('ng20')
X, Y = X_all, Y_all

CPU times: user 33.7 ms, sys: 22.6 ms, total: 56.3 ms
Wall time: 54.7 ms


In [14]:
if False:
    X, Y = X_all[:1000], Y_all[:1000]

In [20]:
%%time
X = preprocessing.preprocess_text_spacy(X, min_length = 2)

CPU times: user 41.6 s, sys: 621 ms, total: 42.2 s
Wall time: 42.4 s


In [17]:
%%time

def get_mats(X, n_jobs = 4, **kwargs):
    return Parallel(n_jobs=n_jobs)(delayed(get_coocurrence_matrix)(d, **kwargs) for d in X)

mats = get_mats(X)

CPU times: user 8.18 s, sys: 651 ms, total: 8.83 s
Wall time: 58.6 s


In [18]:
%%time
graphs = Parallel(n_jobs=4)(delayed(graph_helper.convert_from_numpy_to_nx)(*mat) for mat in mats)

CPU times: user 31.1 s, sys: 1.59 s, total: 32.6 s
Wall time: 34.2 s


In [19]:
%%time
dataset_name = 'ng20-co-occurrence-preprocessed'

def write_mat(idx, graph, label, out_folder):
    filename = os.path.join(out_folder, '{}_{}.gml'.format(label, idx))
    nx.write_gml(graph, filename)
    
def write_mats(mats, Y, dataset_name, n_jobs = 4, out_folder = 'data/graphs/'):
    out_folder = os.path.join(out_folder, dataset_name)
    os.makedirs(out_folder, exist_ok=True)
    Parallel(n_jobs=n_jobs)(delayed(write_mat)(idx, graph, y, out_folder) for idx, (graph, y) in enumerate(zip(graphs, Y)))

num_elements = -1
#write_mats(X[:num_elements], Y[:num_elements], dataset_name)
write_mats(graphs, Y, dataset_name)

CPU times: user 3.95 s, sys: 580 ms, total: 4.53 s
Wall time: 18.1 s
